In [42]:
# Easily Configurable Service Account Key Location
keyLocation = 'resource/local-receiver-341803-261a3b7beea6.json'

In [43]:
import os

from google.cloud import bigquery

# Fetch Credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = keyLocation

# Build BigQuery Client
client = bigquery.Client()

In [44]:
import pandas as pd

# Query for Position Histories from BigQuery
QUERY = ('SELECT lastTrainReporting FROM `local-receiver-341803.testTrainEvents.train-event-data` LIMIT 1')
query_job = client.query(QUERY)

# Wait for job to finish
rows = query_job.result()

# Create a dataframe row holding the identifiable information of the train.
def createTrainIdFrame(trainId):
    # Top level last train reporting
    trainSCAC = trainId.get('trainSCAC')
    # trainSection = trainId.get('trainSection') -- excluding because null
    trainSymbol = trainId.get('trainSymbol')
    # trainDate = trainId.get('trainDate').total_seconds()

    # Construct new dataframe record
    trainIdRow = {'trainSCAC': trainSCAC, 'trainSymbol': trainSymbol }

    return trainIdRow

def createTrainDetailFrame(trainDetail):
    # Top level last train reporting
    maximumTrainSpeed = lastTrainReporting.get('maximumTrainSpeed')
    loadedCarCount = lastTrainReporting.get('loadedCarCount')
    emptyCarCount = lastTrainReporting.get('emptyCarCount')
    trainLength = lastTrainReporting.get('trainLength')
    trailingTonnage = lastTrainReporting.get('trailingTonnage')

    # Construct new dataframe record
    trainDetailRow = {'MaximumTrainSpeed': maximumTrainSpeed, 'LoadedCarCount': loadedCarCount, 
                    'EmptyCarCount': emptyCarCount, 'TrainLength':trainLength, 'TrailingTonnage':trailingTonnage}

    return trainDetailRow

# Create a dataframe from a locomotive detail.
def createLocomotiveFrame(locomotiveDetail):
        return pd.DataFrame.from_dict(locomotiveDetail, orient='index')

def createAotuTrainDataFrame(aotuTrainData):
        return pd.DataFrame.from_dict(aotuTrainData, orient='index')

def createRouteLocationFrame(routeLocation):
    locationId = routeLocation.get('locationId')
    # Milepost location information
    routeSequence = routeLocation.get('routeSequence')
    # The unique identifier of the subdivision
    subdivisionId = routeLocation.get('subdivisionId')
    # The unique identifier fo the milepost
    milepostNumber = routeLocation.get('milepost').get('milepostNumber') 
    # Construct new dataframe record
    dataframeRow = {'LocationId': locationId, 'RouteSequence': routeSequence, 
                    'SubdivisionId': subdivisionId, 'MilepostNumber':milepostNumber}

    return dataframeRow

# Create a dataframe row holding a record of position on the train.
def createPositionHistoryFrame(positionHistory):
    # The time that the train was at the specific milepost
    # Record the position time as seconds from epoch to use in regression model
    positionTime = positionHistory.get('positionTime').timestamp()
    # How fast the train is traveling
    speedMPH = positionHistory.get('speedMPH')
    # Milepost location information
    milepostLocation = positionHistory.get('milepostLocation')
    # The unique identifier of the subdivision
    subdivisionId = milepostLocation.get('subdivisionId')
    # The unique identifier fo the milepost
    milepostNumber = milepostLocation.get('milepost').get('milepostNumber') 

    # Construct new dataframe record
    dataframeRow = {'PositionTime': positionTime, 'SpeedMPH': speedMPH, 
                    'SubdivisionId': subdivisionId, 'MilepostNumber':milepostNumber}
     
    return dataframeRow

for row in rows:
    lastTrainReporting = row.get('lastTrainReporting')
    # Dataframe with the identifable train information
    trainIdFrame = createTrainIdFrame(lastTrainReporting.get('trainId'))
    # Dataframe with the details of the train
    trainDetailFrame = createTrainDetailFrame(lastTrainReporting)
    # Dataframe with the last reported position
    lastReportedPositionFrame = createPositionHistoryFrame(lastTrainReporting.get('lastReportedPosition'))
    # Dataframe for aotu train data
    autoTrainData = lastTrainReporting.get('aotuTrainData')
    # TODO - Fix routeLocationList
    # autoTrainDataFrame = createAotuTrainDataFrame(autoTrainData)

    # Dataframe for Route Locations
    routeDataframe = pd.DataFrame()
    for route in autoTrainData.get('routeLocationList'):
        routeDataframe = routeDataframe.append(dataframeRow, ignore_index = True)

    # Scan the locomotive list
    locomotiveDataFrame = pd.DataFrame()

    for locomotiveDetail in lastTrainReporting.get('locomotiveList'):
        dataframeRow = createLocomotiveFrame(locomotiveDetail)
        # Append the row to the data frame.
        locomotiveDataFrame = locomotiveDataFrame.append(dataframeRow, ignore_index = True)

            # Scan each position history
    positionMilepostTimesDataFrame = pd.DataFrame()

    for positionHistory in lastTrainReporting.get('positionHistoryList'):
        positionDataframeRow = createPositionHistoryFrame(positionHistory)
        #Update the position history frame to include train identifiable information.
        positionDataframeRow.update(trainIdFrame)
        positionDataframeRow.update(trainDetailFrame)

        # Append the row to the data frame.
        positionMilepostTimesDataFrame = positionMilepostTimesDataFrame.append(positionDataframeRow, ignore_index = True)

positionMilepostTimesDataFrame

,EmptyCarCount,LoadedCarCount,MaximumTrainSpeed,MilepostNumber,PositionTime,SpeedMPH,SubdivisionId,TrailingTonnage,TrainLength,trainSCAC,trainSymbol
0,51.0,44.0,50.0,766.9261,1.648865e+09,-11.0,106.0,7586.0,5205.0,UP,WSJGCZ
1,51.0,44.0,50.0,279.0362,1.648864e+09,-10.0,595.0,7586.0,5205.0,UP,WSJGCZ
2,51.0,44.0,50.0,279.1244,1.648864e+09,-9.0,595.0,7586.0,5205.0,UP,WSJGCZ
3,51.0,44.0,50.0,279.2209,1.648864e+09,-7.0,595.0,7586.0,5205.0,UP,WSJGCZ
4,51.0,44.0,50.0,279.3100,1.648864e+09,-3.0,595.0,7586.0,5205.0,UP,WSJGCZ


In [45]:
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import numpy as np

# Fetch the position time
X = np.array(positionMilepostTimesDataFrame.drop('PositionTime',axis=1))
y = np.array(positionMilepostTimesDataFrame.loc[:,'PositionTime']).reshape(5, -1)

# Use Encoder to convert strings to floats
ordinal_encoder = preprocessing.OrdinalEncoder(handle_unknown='use_encoded_value',
                                 unknown_value=-1)
transformed_x = ordinal_encoder.fit_transform(X)

# splitting the data
x_train, x_test, y_train, y_test = train_test_split(transformed_x, y, test_size = 0.2, random_state = 42)

# Fit the linear regression model
regsr=LinearRegression()
regsr.fit(x_train, y_train)

LinearRegression()

In [46]:
y_predicition = regsr.predict(x_test)

m = regsr.coef_
c = regsr.intercept_

print("Predicted y:\n", y_predicition)
print("slope (m): ", m)
print("y-intercept (c): ", c)

Predicted y:
 [[1.64886447e+09]]
slope (m):  [[ 0.00000000e+00  1.77635684e-15  0.00000000e+00 -4.34567901e+00
  -4.71543210e+01 -8.56172840e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
y-intercept (c):  [1.64886452e+09]
